# Hiperparametrização

Escolhemos os parametros de redução de complexidade do modelo, regularização, subamostragem e aprendizado focando em melhorar assertividade do modelo preservando o trade-off entre viés e variância, abaixo detalho os parametros:

1) Redução de complexidade - profundidade das árvores, quanto mais rasa menos propensa a overfitar

2) Regularização L1 para esparsidade

3) Taxa de Aprendizado

4) Redução de fração de amostras na contrução da arvore

Para realizar a troca de `RandomizedSearchCV` por uma busca bayesiana, você pode utilizar a biblioteca `BayesSearchCV` do pacote `scikit-optimize`, que é uma alternativa eficiente para otimização de hiperparâmetros. O principal benefício da busca bayesiana é que ela explora o espaço de parâmetros de forma mais inteligente, ajustando as tentativas com base nos resultados anteriores.
Aqui está o código modificado para usar a busca bayesiana, e com a métrica de recall como critério de avaliação:

### Principais alterações:
1. **Biblioteca usada:** `BayesSearchCV` do `scikit-optimize`.
2. **Espaço de parâmetros:** A sintaxe de intervalos é ligeiramente diferente, com o uso de `(min, max)` e distribuições como `'log-uniform'`.
3. **Critério de avaliação:** O recall foi definido como métrica principal usando `make_scorer(recall_score)` e passado para o parâmetro `scoring`.
4. **Parâmetro `n_estimators`:** O intervalo foi ampliado para permitir uma busca mais ampla.
Essa abordagem permitirá uma busca mais eficiente, focando em otimizar o recall.

In [ ]:
import xgboost as xgb
from sklearn.metrics import make_scorer, recall_score
from skopt import BayesSearchCV
from xgboost import XGBClassifier

# Definir o modelo inicial
model = XGBClassifier(scale_pos_weight=scale_pos_weight, use_label_encoder=False)

# Parâmetros para Busca Bayesiana (extremamente simplificados)
param_dist = {
    'max_depth': (1, 2),  # Profundidade mínima
    'reg_alpha': (0.01, 0.1),  # Regularização leve
    'learning_rate': (0.01, 0.1),  # Taxa de aprendizado baixa
    'n_estimators': (5, 10),  # Valor muito baixo para árvores
    'subsample': (0.5, 1.0),  # Usar entre 50% a 100% dos dados
    'colsample_bytree': (0.5, 1.0)  # Usar entre 50% a 100% das colunas
}

# Scorer baseado em recall
recall = make_scorer(recall_score)

# Amostragem do conjunto de dados (1% dos dados)
X_train_sample = X_train.sample(frac=0.01, random_state=42)  # Usar 1% dos dados
y_train_sample = y_train.loc[X_train_sample.index]

# BayesSearchCV com validação cruzada (configurado para ser extremamente rápido)
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_dist,
    n_iter=1,  # Apenas uma iteração
    cv=2,  # Validação cruzada com 2 folds
    n_jobs=1,  # Usar apenas um núcleo para evitar sobrecarga
    verbose=0,  # Silenciar logs
    random_state=3,
    scoring=recall
)

# Aplicando a busca (sem avaliação)
bayes_search.fit(X_train_sample, y_train_sample)


In [ ]:
# Melhor modelo e parâmetros
best_model_bayes = bayes_search.best_estimator_
print(f"Best parameters (BayesSearchCV): {bayes_search.best_params_}")

In [ ]:
print(f"Best score (BayesSearchCV - Recall): {bayes_search.best_score_}")

In [ ]:
# Avaliação do modelo final
y_pred_bayes = best_model_bayes.predict(X_test)
y_pred_proba_bayes = best_model_bayes.predict_proba(X_test)[:, 1]

# Métricas
accuracy_bayes = recall_score(y_test, y_pred_bayes)
auc_bayes = roc_auc_score(y_test, y_pred_proba_bayes)
f1_bayes = f1_score(y_test, y_pred_bayes)

In [ ]:
print(f"Test recall (BayesSearchCV): {accuracy_bayes}")
print(f"Test AUC (BayesSearchCV): {auc_bayes}")
print(f"Test F1 Score (BayesSearchCV): {f1_bayes}")